In [20]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import (
    JsonOutputParser,
)
from langchain.tools import (
    Tool,
)
from langchain.chains import LLMMathChain
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import END, START, StateGraph
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
from typing import TypedDict, Optional, List, Tuple, Literal, Dict
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from IPython.display import Image, display


In [2]:
#%%sh 

#pip install -r ../requirements.txt

In [3]:
# Load environment variables from the .env file
load_dotenv("../config/dev.env")
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=OPENAPI_KEY, model="gpt-4o", temperature=0.0)
#llm.invoke("heelo")

In [22]:
from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate

question = "there were 10 apples in a basket. 5 apples were taken out. How many apples are left in the basket?"
possible_answers = ["5", "10", "15", "0"]

word_problem_template = """You are a teacher developing exam questions for students. 
You have this question {question} that you want to ask your students and these four possible answers {possible_answers}.
You need to validate that each possible answer and return if the answer is correct or incorrect."""

math_assistant_prompt = PromptTemplate(input_variables=["question"],
                                       template=word_problem_template
                                       )
llm_math = LLMMathChain.from_llm(llm=llm)

word_problem_tool = Tool(name="MathReasoningTool",
                                       func=llm_math.run,
                                       description="A tool that helps you solve logic-based questions",
                                    )
tools = [word_problem_tool]
llm_with_tools = llm.bind_tools(tools=tools)

class MathQuestion(BaseModel):
   answer_1: bool = Field(description="Is the first answer correct?")
   answer_2: bool = Field(description="Is the second answer correct?")
   answer_3: bool = Field(description="Is the third answer correct?")
   answer_4: bool = Field(description="Is the fourth answer correct?")


prompt = word_problem_template.format(question=question, possible_answers=possible_answers)
#rsp = llm_with_tools.invoke([prompt])
rsp = llm_with_tools.with_structured_output(MathQuestion).invoke([prompt])
rsp

MathQuestion(answer_1=True, answer_2=False, answer_3=False, answer_4=False)

In [12]:
from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from typing import List, Dict, Any
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class MathProblemSolver:
    def __init__(self, model_name: str = "gpt-4", temperature: float = 0.0):
        """
        Initialize the Math Problem Solver with necessary components
        
        Args:
            model_name (str): The OpenAI model to use (default: "gpt-4")
            temperature (float): The temperature setting for the model (default: 0.0)
        """
        # Get API key from environment variables
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY not found in environment variables")

        # Initialize the LLM (fixed model name from "gpt-4o" to "gpt-4")
        self.llm = ChatOpenAI(
            api_key=self.api_key,
            model_name=model_name,
            temperature=temperature
        )

        # Create the prompt template (fixed indentation)
        self.word_problem_template = """You are a reasoning agent tasked with solving 
the user's logic-based questions. Logically arrive at the solution, and be 
factual. In your answers, clearly detail the steps involved and give the 
final answer. Provide the response in bullet points.

Question: {question}
Answer:"""
        
        self.math_assistant_prompt = PromptTemplate(
            input_variables=["question"],
            template=self.word_problem_template
        )
        
        # Initialize the math chain
        self.llm_math = LLMMathChain.from_llm(llm=self.llm)
        
        # Create the reasoning tool
        self.word_problem_tool = Tool(
            name="Reasoning Tool",
            func=self.llm_math.run,
            description="A tool that helps you solve logic-based questions"
        )
        
        # Bind tools to the LLM
        self.llm_with_tools = self.llm.bind_tools([self.word_problem_tool])
    
    def solve_problem(self, question: str) -> Dict[str, Any]:
        """
        Solve a given math problem
        
        Args:
            question (str): The math problem to solve
            
        Returns:
            Dict[str, Any]: Response containing the solution and metadata
        """
        if not isinstance(question, str) or not question.strip():
            return {
                "status": "error",
                "question": question,
                "solution": None,
                "error": "Invalid question format or empty question"
            }

        try:
            # Format the prompt
            prompt = self.math_assistant_prompt.format(question=question)
            
            # Get the response
            response = self.llm_with_tools.invoke(prompt)
            
            return {
                "status": "success",
                "question": question,
                "solution": response,
                "error": None
            }
            
        except Exception as e:
            return {
                "status": "error",
                "question": question,
                "solution": None,
                "error": str(e)
            }
    
    def solve_multiple_problems(self, questions: List[str]) -> List[Dict[str, Any]]:
        """
        Solve multiple math problems
        
        Args:
            questions (List[str]): List of math problems to solve
            
        Returns:
            List[Dict[str, Any]]: List of responses for each problem
        """
        if not isinstance(questions, list):
            return [{
                "status": "error",
                "question": None,
                "solution": None,
                "error": "Input must be a list of questions"
            }]
            
        return [self.solve_problem(q) for q in questions]

def main():
    """
    Main function to demonstrate the usage of MathProblemSolver
    """
    try:
        # Initialize the solver
        solver = MathProblemSolver()
        
        # Single problem
        question = "There were 10 apples in a basket. 5 apples were taken out. How many apples are left in the basket?"
        result = solver.solve_problem(question)
        print("\nSingle Problem Result:")
        print(result)
        
        # Multiple problems
        questions = [
            "There were 10 apples in a basket. 5 apples were taken out. How many apples are left in the basket?",
            "If John has 15 marbles and gives 3 to Mary, how many does he have left?"
        ]
        results = solver.solve_multiple_problems(questions)
        print("\nMultiple Problems Results:")
        for result in results:
            print(result)
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

An error occurred: 


In [13]:
class GraphState(TypedDict):
    init_input: Optional[str]
    fruit: Optional[str]
    final_result: Optional[str]
    ai_confirmation: Optional[bool]
    revision_count: Optional[int]
    init_input_length: Optional[int]
    output_length: Optional[int]

def input_fruit(state: GraphState) -> GraphState:
    print("Node: input_fruit")
    init_input = state.get("init_input", "").strip().lower()
    state["fruit"] = init_input
    return state

def review_fruit(state: GraphState) -> GraphState:
    print("--------------------")
    print("Node: review_fruit")
    print(f"Review your selection: {state['fruit']}, is this correct?")

    class ValidFruit(BaseModel):
        fruit: str = Field(..., title="Fruit", description="The fruit you selected")
        valid_fruit: bool = Field(..., title="Valid Fruit", description="Is the fruit valid?")

    message = HumanMessage(content=[
        {"type": "text", "text": f"Review your selection: {state['fruit']}, is this correct?"}
    ])
    response = llm.with_structured_output(ValidFruit).invoke([message])
    valid_fruit = response.valid_fruit

    if valid_fruit:
        state["ai_confirmation"] = True
    else:
        state["ai_confirmation"] = False

    return state

In [6]:
def rename_fruit(state: GraphState) -> GraphState:
    print("--------------------")
    print("Node: rename_fruit")
    print(f"Rename the fruit: {state['fruit']} to an actual fruit")

    # appending old attempts to rename the fruit that did not pass ai_confirmation
    if state.get("revision_count", 0) > 0:
        previous_attempt = AIMessage(content=f"Rename the fruit: {state['fruit']} to an actual fruit")
        correction_feedback = AIMessage(content=f"Confirmation if correct{state['ai_confirmation']}")
    else:
        print("No previous attempts to rename the fruit")

    class FruitRename(BaseModel):
        fruit: str = Field(..., title="Fruit", description="The fruit you selected")

    system_message = SystemMessage(content=f"""Rename the fruit: {state['fruit']} to an actual fruit.  Use aggressive renaming and trying differernt letter in the
                                   alphabet to make a valid fruit.
                                   RULES:
                                   1. You must use at least the first letter of the fruit you selected.
                                   2. The output must be a valid fruit.
                                   3. Do not use the same answer as before.""")
    if state.get("revision_count", 0) > 0:
        messages = [previous_attempt, correction_feedback, system_message]
    else:
        messages = [system_message]
    response = llm.with_structured_output(FruitRename).invoke(messages)
    print('fixing fruit response:', response)
    state["fruit"] = response.fruit
    revision_count = state.get("revision_count", 0)
    state["revision_count"] = revision_count + 1

    return state

def review_decision(state: GraphState) -> Tuple[Literal["summarize_output", "rename_fruit"]]:
    print("--------------------")
    print("Function Edge: review_decision")
    print(f"Function Edge Contine: state: {state}")

    if state.get("ai_confirmation") == True:
        print("Function Edge: reveiw_decision -> summarize_output")
        return "summarize_output"
    elif state.get("ai_confirmation") == False:
        print("Function Edge Continue: to_error")
        return "rename_fruit"

In [27]:
# workflow.add_node("summarize_output", action=summarize_output)
def summarize_output(state: GraphState) -> Dict[str, str]:
    print("--------------------")
    print("Node: summarize_output")
    
    if state.get("ai_confirmation") and (state.get("init_input") != state.get("fruit")):
        return {"final_result": f"you entered {state['init_input']} and the AI corrected it to {state['fruit']}"}
    elif state.get("ai_confirmation") and (state.get("init_input") == state.get("fruit")):
        return {"final_result": f"you entered {state['init_input']} and the AI confirmed it"}
    elif state.get("ai_confirmation") == False:
        return {"final_result": f"you entered {state['init_input']} and the AI could not correct it"}

MAX_REVISIONS = 3
def extract_or_end(state: GraphState) -> Tuple[Literal["review_fruit", "summarize_output"]]:
    if state.get("revision_count", 0) >= MAX_REVISIONS:
        return "summarize_output"
    else:
        return "review_fruit"
    
def word_problem_tool(state: GraphState) -> Dict[str, str]:
    print("--------------------")
    print("Node: math_tool_llm")

    problem_chain = LLMMathChain.from_llm(llm=llm)
    math_tool = Tool.from_function(
        name="Calculator",
        func=problem_chain.run,
        description="Useful for when you need to answer questions about math. Only use this tool with numbers. This tool is only for math questions and nothing else. Only input math expressions.",
    )

    word_problem_template = """You are a reasoning agent tasked with solving 
    the user's logic-based questions. Logically arrive at the solution, and be 
    factual. In your answers, clearly detail the steps involved and give the 
    final answer. Provide the response in bullet points. 
    Question: {question} 
    Answer:"""

    math_assistant_prompt = PromptTemplate(
        input_variables=["question"], template=word_problem_template
    )

    word_problem_chain = LLMChain(llm=llm, prompt=math_assistant_prompt)
    word_problem_tool = Tool.from_function(
        name="Reasoning Tool",
        func=word_problem_chain.run,
        description="Useful for when you need to answer logic-based/reasoning questions.",
    )

    init_input = state.get("init_input", "").strip().lower()
    state["init_input"] = init_input
    len_input = len(init_input)
    
    # Use Calculator tool
    question = f"What is {len_input} + 1?"
    result = problem_chain.run(question)
    
    # Update state with results
    state["init_input"] = str(result)
    state["init_input_length"] = len_input
    state["output_length"] = int(result)
    
    return {"init_input": state["init_input"], "init_input_length": len_input, "output_length": int(result)}

In [28]:
workflow = StateGraph(GraphState)
workflow.set_entry_point("input_fruit")
workflow.add_node("input_fruit", action=input_fruit)
workflow.add_node("review_fruit", action=review_fruit)
workflow.add_node("rename_fruit", action=rename_fruit)
workflow.add_node("summarize_output", action=summarize_output)
workflow.add_node("math_tool_llm", action=word_problem_tool)

workflow.add_edge("input_fruit", "review_fruit")
workflow.add_edge("summarize_output", "math_tool_llm")
workflow.add_edge("math_tool_llm", END)

workflow.add_conditional_edges(
    source="review_fruit",
    path=review_decision,
    path_map={
        "summarize_output": "summarize_output",
        "rename_fruit": "rename_fruit",
    },
)
workflow.add_conditional_edges(
    source="rename_fruit",
    path=extract_or_end,
    path_map={
        "review_fruit": "review_fruit",
        "summarize_output": "summarize_output",
    },
)


app = workflow.compile()

# Test with a valid fruit
result = app.invoke({"init_input": "ap"})
print("-------")
for k, v in result.items():
    print(f"{k}: {v}")

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

Node: input_fruit
--------------------
Node: review_fruit
Review your selection: ap, is this correct?
--------------------
Function Edge: review_decision
Function Edge Contine: state: {'init_input': 'ap', 'fruit': 'ap', 'ai_confirmation': False}
Function Edge Continue: to_error
--------------------
Node: rename_fruit
Rename the fruit: ap to an actual fruit
No previous attempts to rename the fruit
fixing fruit response: fruit='apple'
--------------------
Node: review_fruit
Review your selection: apple, is this correct?
--------------------
Function Edge: review_decision
Function Edge Contine: state: {'init_input': 'ap', 'fruit': 'apple', 'ai_confirmation': True, 'revision_count': 1}
Function Edge: reveiw_decision -> summarize_output
--------------------
Node: summarize_output
--------------------
Node: math_tool_llm


/tmp/ipykernel_76006/2652618489.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = problem_chain.run(question)


ValueError: invalid literal for int() with base 10: 'Answer: 3'